In [1]:
import numpy as np
import pandas as pd
from tree_features import connectome_average, matrix_fusion, tree_connectivity, tree_dictionary
from io_functions import load_data
import os
import json
import glob
import inspect


In [2]:
project_path = "/home/antoniojm/Documents/projects/BHA_5G/lemon"
sc_group = load_data(os.path.join(project_path, "sc"))
fc_group = load_data(os.path.join(project_path, "fc"))
fcm, scm = connectome_average(fc_group, sc_group)

for g in np.arange(0, 1.1, 0.1):
    W = matrix_fusion(g, fcm, scm)
    t_dict = tree_dictionary(10, 20, W)
    json.dump(t_dict, open(os.path.join(project_path, "df_connectomes", "tree_features_g_" + str(g) + ".json"), "w"))
    full_df = pd.DataFrame()
    for idx in range(0,len(sc_group)):
        sc_indiv = pd.DataFrame(sc_group[idx])
        fc_indiv = pd.DataFrame(fc_group[idx])
        t_features = tree_connectivity(10, 20, W, sc_indiv, fc_indiv)
        full_df = full_df.append(t_features)
    
    full_df = full_df.loc[:, ~full_df.apply(lambda x: x.duplicated(), axis=1).all()].copy()
    full_df.to_csv(os.path.join(project_path, "df_connectomes", "tree_features_g_" + str(g) + ".csv"), index=False)
